# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 16 2022 9:41AM,244242,10,BPS20023,Beach Patient Assistance,Released
1,Aug 16 2022 9:37AM,244241,10,SOTAH0000755,"Nextsource Biotechnology, LLC",Released
2,Aug 16 2022 9:30AM,244240,10,CLI00356.1,"CLINUVEL, Inc.",Released
3,Aug 16 2022 9:30AM,244215,15,30009949,"Alliance Pharma, Inc.",Released
4,Aug 16 2022 9:30AM,244215,15,30009950,"Alliance Pharma, Inc.",Released
5,Aug 16 2022 9:30AM,244215,15,30009954,"Alliance Pharma, Inc.",Released
6,Aug 16 2022 9:30AM,244215,15,30009953,"Alliance Pharma, Inc.",Released
7,Aug 16 2022 9:30AM,244215,15,30009951,"Alliance Pharma, Inc.",Released
8,Aug 16 2022 9:30AM,244215,15,30009955,"Alliance Pharma, Inc.",Released
9,Aug 16 2022 9:30AM,244215,15,30009952,"Alliance Pharma, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
31,244238,Released,3
32,244239,Released,2
33,244240,Released,1
34,244241,Released,1
35,244242,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
244238,NaN,NaN,3.0
244239,NaN,NaN,2.0
244240,NaN,NaN,1.0
244241,NaN,NaN,1.0
244242,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
244147,0.0,16.0,29.0
244155,0.0,1.0,0.0
244164,0.0,0.0,1.0
244183,0.0,0.0,1.0
244187,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
244147,0,16,29
244155,0,1,0
244164,0,0,1
244183,0,0,1
244187,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,244147,0,16,29
1,244155,0,1,0
2,244164,0,0,1
3,244183,0,0,1
4,244187,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,244147,,16,29
1,244155,,1,
2,244164,,,1
3,244183,,,1
4,244187,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 16 2022 9:41AM,244242,10,Beach Patient Assistance
1,Aug 16 2022 9:37AM,244241,10,"Nextsource Biotechnology, LLC"
2,Aug 16 2022 9:30AM,244240,10,"CLINUVEL, Inc."
3,Aug 16 2022 9:30AM,244215,15,"Alliance Pharma, Inc."
77,Aug 16 2022 9:27AM,244216,15,"Brookfield Pharmaceuticals, LLC"
115,Aug 16 2022 9:25AM,244239,10,"Amcyte Pharma, Inc."
117,Aug 16 2022 9:21AM,244237,12,Hush Hush
127,Aug 16 2022 9:21AM,244238,10,Beach Products Inc
130,Aug 16 2022 8:49AM,244236,10,"Methapharm, Inc."
138,Aug 16 2022 8:47AM,244235,10,Emerginnova


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Aug 16 2022 9:41AM,244242,10,Beach Patient Assistance,,,1
1,Aug 16 2022 9:37AM,244241,10,"Nextsource Biotechnology, LLC",,,1
2,Aug 16 2022 9:30AM,244240,10,"CLINUVEL, Inc.",,,1
3,Aug 16 2022 9:30AM,244215,15,"Alliance Pharma, Inc.",,,74
4,Aug 16 2022 9:27AM,244216,15,"Brookfield Pharmaceuticals, LLC",,,38
5,Aug 16 2022 9:25AM,244239,10,"Amcyte Pharma, Inc.",,,2
6,Aug 16 2022 9:21AM,244237,12,Hush Hush,,,10
7,Aug 16 2022 9:21AM,244238,10,Beach Products Inc,,,3
8,Aug 16 2022 8:49AM,244236,10,"Methapharm, Inc.",,,8
9,Aug 16 2022 8:47AM,244235,10,Emerginnova,,,3


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 16 2022 9:41AM,244242,10,Beach Patient Assistance,1,,
1,Aug 16 2022 9:37AM,244241,10,"Nextsource Biotechnology, LLC",1,,
2,Aug 16 2022 9:30AM,244240,10,"CLINUVEL, Inc.",1,,
3,Aug 16 2022 9:30AM,244215,15,"Alliance Pharma, Inc.",74,,
4,Aug 16 2022 9:27AM,244216,15,"Brookfield Pharmaceuticals, LLC",38,,
5,Aug 16 2022 9:25AM,244239,10,"Amcyte Pharma, Inc.",2,,
6,Aug 16 2022 9:21AM,244237,12,Hush Hush,10,,
7,Aug 16 2022 9:21AM,244238,10,Beach Products Inc,3,,
8,Aug 16 2022 8:49AM,244236,10,"Methapharm, Inc.",8,,
9,Aug 16 2022 8:47AM,244235,10,Emerginnova,3,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 16 2022 9:41AM,244242,10,Beach Patient Assistance,1,,
1,Aug 16 2022 9:37AM,244241,10,"Nextsource Biotechnology, LLC",1,,
2,Aug 16 2022 9:30AM,244240,10,"CLINUVEL, Inc.",1,,
3,Aug 16 2022 9:30AM,244215,15,"Alliance Pharma, Inc.",74,,
4,Aug 16 2022 9:27AM,244216,15,"Brookfield Pharmaceuticals, LLC",38,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 16 2022 9:41AM,244242,10,Beach Patient Assistance,1.0,NaN,NaN
1,Aug 16 2022 9:37AM,244241,10,"Nextsource Biotechnology, LLC",1.0,NaN,NaN
2,Aug 16 2022 9:30AM,244240,10,"CLINUVEL, Inc.",1.0,NaN,NaN
3,Aug 16 2022 9:30AM,244215,15,"Alliance Pharma, Inc.",74.0,NaN,NaN
4,Aug 16 2022 9:27AM,244216,15,"Brookfield Pharmaceuticals, LLC",38.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 16 2022 9:41AM,244242,10,Beach Patient Assistance,1.0,0.0,0.0
1,Aug 16 2022 9:37AM,244241,10,"Nextsource Biotechnology, LLC",1.0,0.0,0.0
2,Aug 16 2022 9:30AM,244240,10,"CLINUVEL, Inc.",1.0,0.0,0.0
3,Aug 16 2022 9:30AM,244215,15,"Alliance Pharma, Inc.",74.0,0.0,0.0
4,Aug 16 2022 9:27AM,244216,15,"Brookfield Pharmaceuticals, LLC",38.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3663388,138.0,0.0,0.0
12,244237,10.0,0.0,0.0
15,488431,112.0,0.0,0.0
16,976797,4.0,0.0,0.0
19,976750,6.0,25.0,1.0
20,732556,31.0,16.0,0.0
21,976779,4.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3663388,138.0,0.0,0.0
1,12,244237,10.0,0.0,0.0
2,15,488431,112.0,0.0,0.0
3,16,976797,4.0,0.0,0.0
4,19,976750,6.0,25.0,1.0
5,20,732556,31.0,16.0,0.0
6,21,976779,4.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,138.0,0.0,0.0
1,12,10.0,0.0,0.0
2,15,112.0,0.0,0.0
3,16,4.0,0.0,0.0
4,19,6.0,25.0,1.0
5,20,31.0,16.0,0.0
6,21,4.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,138.0
1,12,Released,10.0
2,15,Released,112.0
3,16,Released,4.0
4,19,Released,6.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Completed,0.0,0.0,0.0,0.0,1.0,0.0,0.0
Executing,0.0,0.0,0.0,0.0,25.0,16.0,0.0
Released,138.0,10.0,112.0,4.0,6.0,31.0,4.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Completed,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,Executing,0.0,0.0,0.0,0.0,25.0,16.0,0.0
2,Released,138.0,10.0,112.0,4.0,6.0,31.0,4.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Completed,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,Executing,0.0,0.0,0.0,0.0,25.0,16.0,0.0
2,Released,138.0,10.0,112.0,4.0,6.0,31.0,4.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()